In [2]:
import hyperopt
from hyperopt import fmin, tpe, hp
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential

2023-09-03 22:24:40.247060: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-03 22:24:40.438090: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-03 22:24:40.440037: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-03 22:24:41.521498: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
# Загрузка и предобработка данных
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
def objective(params):
    model = Sequential([
        Flatten(input_shape=(28, 28, 1)),
        Dense(params['units'], activation='relu'),
        Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), verbose=0)
    val_accuracy = history.history['val_accuracy'][-1]
    
    return -val_accuracy  # Минимизация, так как Hyperopt ищет минимум функции

In [5]:
space = {
    'units': hp.choice('units', [32, 64, 128]),  # Количество нейронов в слое
    'learning_rate': hp.loguniform('learning_rate', -5, -1)  # Логарифмический выбор скорости обучения
}

In [6]:
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)  # Количество экспериментов
# на 7 эпохе достигла минимума и не менялась

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2023-09-03 22:25:41.665922: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.
2023-09-03 22:25:47.308040: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 31360000 exceeds 10% of free system memory.


 10%|█         | 1/10 [00:54<08:08, 54.23s/trial, best loss: -0.751800000667572]

2023-09-03 22:26:35.533873: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.
2023-09-03 22:26:39.947151: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 31360000 exceeds 10% of free system memory.


 20%|██        | 2/10 [01:40<06:38, 49.80s/trial, best loss: -0.9587000012397766]

2023-09-03 22:27:21.959521: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


100%|██████████| 10/10 [07:56<00:00, 47.61s/trial, best loss: -0.9587000012397766]


In [22]:
import numpy as np
from sklearn.datasets import make_classification
from tensorflow import keras

from scikeras.wrappers import KerasClassifier


#X, y = make_classification(1000, 20, n_informative=10, random_state=0)
x_train = x_train.astype(np.float32)
x_train = np.reshape(x_train, [60000,784])
x_test = x_test.astype(np.float32)
x_test = np.reshape(x_test, [10000,784])
y_train = y_train.astype(np.int64)
y_train = y_train.astype(np.int64)


def get_model(hidden_layer_dim, meta):
    # note that meta is a special argument that will be
    # handed a dict containing input metadata
    n_features_in_ = meta["n_features_in_"]
    X_shape_ = meta["X_shape_"]
    n_classes_ = meta["n_classes_"]

    model = keras.models.Sequential()
    model.add(keras.layers.Dense(n_features_in_, input_shape=X_shape_[1:]))
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.Dense(hidden_layer_dim))
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.Dense(n_classes_))
    model.add(keras.layers.Activation("softmax"))
    return model

clf = KerasClassifier(
    get_model,
    loss="sparse_categorical_crossentropy",
    hidden_layer_dim=100,
)

clf.fit(x_train, y_train)
y_proba = clf.predict_proba(x_train)


1875/1875 [==============================] - 5s 3ms/step


In [24]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


pipe = Pipeline([
    ('scale', StandardScaler()),
    ('clf', clf),
])

pipe.fit(x_train, y_train)
y_proba = pipe.predict_proba(x_test)

313/313 [==============================] - 1s 3ms/step


In [25]:
from sklearn.model_selection import GridSearchCV


params = {
    "hidden_layer_dim": [50, 100, 200],
    "loss": ["sparse_categorical_crossentropy"],
    "optimizer": ["adam", "sgd"],
    "optimizer__learning_rate": [0.0001, 0.001, 0.1],
}
gs = GridSearchCV(clf, params, refit=False, cv=3, scoring='accuracy')

gs.fit(x_train, y_train)
print(gs.best_score_, gs.best_params_)

625/625 [==============================] - 2s 3ms/step
0.9620000000000001 {'hidden_layer_dim': 50, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'adam', 'optimizer__learning_rate': 0.001}


In [34]:
from tensorflow.keras import layers, models, optimizers, losses
model_optimum = keras.models.Sequential()
model_optimum.add(keras.layers.Dense(50, input_shape=x_train.shape[1:]))
model_optimum.add(keras.layers.Activation("relu"))
model_optimum.add(keras.layers.Dense(50))
model_optimum.add(keras.layers.Activation("relu"))
model_optimum.add(keras.layers.Dense(10))
model_optimum.add(keras.layers.Activation("softmax"))

model_optimum.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss=losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model_optimum.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2988 - accuracy: 0.9145
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1377 - accuracy: 0.9589
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1035 - accuracy: 0.9688
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0840 - accuracy: 0.9747
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0715 - accuracy: 0.9783
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0601 - accuracy: 0.9811
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0523 - accuracy: 0.9829
Epoch 8/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0465 - accuracy: 0.9847
Epoch 9/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0407 - accuracy: 0.9864
Epoch 10/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.036

In [36]:
test_loss, test_acc = model_optimum.evaluate(x_test, y_test)
print(f"Accuracy on the test set: {test_acc * 100:.2f}%")

313/313 [==============================] - 1s 2ms/step - loss: 0.0976 - accuracy: 0.9733
Accuracy on the test set: 97.33%


In [37]:
from sklearn.model_selection import RandomizedSearchCV

params = {
    "hidden_layer_dim": [50, 100, 200],
    "loss": ["sparse_categorical_crossentropy"],
    "optimizer": ["adam", "sgd"],
    "optimizer__learning_rate": [0.0001, 0.001, 0.1],
}
gs = RandomizedSearchCV(clf, params, refit=False, cv=3, scoring='accuracy')

gs.fit(x_test, y_test)
print(gs.best_score_, gs.best_params_)

105/105 [==============================] - 0s 3ms/step
0.9094014080472235 {'optimizer__learning_rate': 0.001, 'optimizer': 'adam', 'loss': 'sparse_categorical_crossentropy', 'hidden_layer_dim': 100}


In [38]:
from tensorflow.keras import layers, models, optimizers, losses
model_optimum = keras.models.Sequential()
model_optimum.add(keras.layers.Dense(100, input_shape=x_train.shape[1:]))
model_optimum.add(keras.layers.Activation("relu"))
model_optimum.add(keras.layers.Dense(100))
model_optimum.add(keras.layers.Activation("relu"))
model_optimum.add(keras.layers.Dense(100))
model_optimum.add(keras.layers.Activation("softmax"))

model_optimum.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss=losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model_optimum.fit(x_train, y_train, epochs=10, batch_size=32)
test_loss, test_acc = model_optimum.evaluate(x_test, y_test)
print(f"Accuracy on the test set: {test_acc * 100:.2f}%")

Epoch 1/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2933 - accuracy: 0.9152
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1181 - accuracy: 0.9648
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0827 - accuracy: 0.9741
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0617 - accuracy: 0.9810
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0488 - accuracy: 0.9843
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0378 - accuracy: 0.9879
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0343 - accuracy: 0.9882
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0279 - accuracy: 0.9913
Epoch 9/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0238 - accuracy: 0.9922
Epoch 10/10
313/313 [==============================] - 1s 2ms/step - loss: 0.1017 

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from bayes_opt import BayesianOptimization

# Определение функции для оптимизации
def keras_cnn(learning_rate, conv_1_filter, conv_1_kernel,
              num_conv_layers, dense_units):
    model = tf.keras.Sequential()
    model.add(Conv2D(filters=int(conv_1_filter), kernel_size=int(conv_1_kernel),
                     activation='relu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    for i in range(int(num_conv_layers)):
        model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(units=int(dense_units), activation='relu'))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train, epochs=5, validation_split=0.1, verbose=0)
    val_accuracy = history.history['val_accuracy'][-1]
    return val_accuracy

# Определение диапазонов гиперпараметров для оптимизации
pbounds = {'learning_rate': (1e-4, 1e-2),
           'conv_1_filter': (32, 128),
           'conv_1_kernel': (3, 5),
           'num_conv_layers': (1, 3),
           'dense_units': (32, 512)}

# Создание объекта BayesianOptimization
optimizer = BayesianOptimization(f=keras_cnn, pbounds=pbounds, verbose=2, random_state=1)

# Запуск оптимизации
optimizer.maximize(init_points=5, n_iter=10)

# Вывод результатов оптимизации
print("Best parameters:", optimizer.max['params'])
print("Best validation accuracy:", optimizer.max['target'])

2023-09-03 23:50:42.797748: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-03 23:50:42.848201: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-03 23:50:42.848993: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-03 23:50:43.807704: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


ModuleNotFoundError: No module named 'bayes_opt'

In [19]:
x_train.shape

(60000, 784)